In [1]:
import gym
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from memory import replayBuffer

In [2]:
def  Neural_Network():
    state_input=tf.keras.layers.Input((4,),name='state_input')
    action_input=tf.keras.layers.Input((2,),name='action_input')

    net=tf.keras.layers.Dense(256,'relu')(state_input)
    net=tf.keras.layers.Dense(256,'relu')(net)
    net=tf.keras.layers.Dense(2)(net)

    Q_vals=tf.multiply(net,action_input,name='masking')

    model=tf.keras.Model(inputs=[state_input,action_input],outputs=[Q_vals],name='DQN')
    model.compile(loss='mse',optimizer=tf.keras.optimizers.RMSprop(lr=0.00025, rho=0.95, epsilon=0.01))
    return model

In [3]:
dqn=Neural_Network()
fixed_dqn=Neural_Network()

In [4]:
def hard_copy(source,target):
    target.set_weights(source.get_weights())
    #for svar,tvar in zip(source.variables,target.variables):
     #   tvar.assign(svar,read_value=False)

In [5]:
def train_on_batch(s,a,r,s_,nd):
    batch_size=s.shape[0]
    q_targets=r+0.99*np.multiply(nd,np.max(fixed_dqn.predict([s_,np.ones((batch_size,2))]),axis=1))
    q_target_formated=np.multiply(q_targets.reshape(-1,1),tf.keras.utils.to_categorical(a,2))
    
    dqn.train_on_batch([s,tf.keras.utils.to_categorical(a,2)],q_target_formated)

In [6]:
def get_episilon(step,steady_episilon=0.01,steady_step=100000):
    if step>steady_step:
        return steady_episilon
    else:
        m=(steady_episilon-1)/steady_step
        return m*step+1

In [7]:
def getAction(s,episilon):
    if np.random.random()<episilon:
        return np.random.randint(2)
    else:
        optimal_action=np.argmax(dqn.predict([s.reshape(1,-1),np.ones((1,2))]),axis=1)[0]
        return optimal_action

In [8]:
#Training

In [9]:
env=gym.make('CartPole-v0')
env._max_episode_steps=200
memory=replayBuffer(100000)

In [10]:
curr_state=env.reset()
for _ in range(50000):
    act=env.action_space.sample()
    next_state,reward,done,info=env.step(act)
    memory.push(curr_state,act,reward,next_state,not done)
    
    if done:
        curr_state=env.reset()
    else:
        curr_state=next_state

In [11]:
episode_reward=0
curr_state=env.reset()

for step in range(1000000):
    act=getAction(curr_state,get_episilon(step,steady_episilon=0.01, steady_step=100000))
    next_state,reward,done,info=env.step(act)
    episode_reward+=reward
    
    memory.push(curr_state,act,reward,next_state,not done)
    
    if done:
        curr_state=env.reset()
        print('On step {} episode reward {}'.format(step,episode_reward))
        episode_reward=0
    else:
        curr_state=next_state
        
    if step%10000==0:
        hard_copy(dqn,fixed_dqn)
        
    if step%4==0:
        s,a,r,s_,nd=memory.sample(32)
        train_on_batch(s,a,r,s_,nd)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
On step 14 episode reward 15.0
On step 29 episode reward 15.0
On step 50 episode reward 21.0
On step 59 episode reward 9.0
On step 92 episode reward 33.0
On step 101 episode reward 9.0
On step 113 episode reward 12.0
On step 143 episode reward 30.0
On step 166 episode reward 23.0
On step 176 episode reward 10.0
On step 213 episode reward 37.0
On step 237 episode reward 24.0
On step 259 episode reward 22.0
On step 284 episode reward 25.0
On step 309 episode reward 25.0
On step 353 episode reward 44.0
On step 365 episode reward 12.0
On step 378 episode reward 13.0
On step 398 episode reward 20.0
On step 416 episode reward 18.0
On step 425 episode reward 9.0
On step 436 episode reward 11.0
On step 455 episode reward 19.0
On step 470 episode reward 15.0
On step 493 episode reward 23.0
On step 504 episode reward 11.0
On step 518 episode reward 14.0
On step 542 episode reward 

On step 5011 episode reward 12.0
On step 5022 episode reward 11.0
On step 5033 episode reward 11.0
On step 5056 episode reward 23.0
On step 5080 episode reward 24.0
On step 5095 episode reward 15.0
On step 5111 episode reward 16.0
On step 5122 episode reward 11.0
On step 5140 episode reward 18.0
On step 5148 episode reward 8.0
On step 5163 episode reward 15.0
On step 5175 episode reward 12.0
On step 5210 episode reward 35.0
On step 5226 episode reward 16.0
On step 5237 episode reward 11.0
On step 5248 episode reward 11.0
On step 5296 episode reward 48.0
On step 5317 episode reward 21.0
On step 5355 episode reward 38.0
On step 5400 episode reward 45.0
On step 5414 episode reward 14.0
On step 5432 episode reward 18.0
On step 5449 episode reward 17.0
On step 5471 episode reward 22.0
On step 5495 episode reward 24.0
On step 5568 episode reward 73.0
On step 5580 episode reward 12.0
On step 5598 episode reward 18.0
On step 5627 episode reward 29.0
On step 5647 episode reward 20.0
On step 566

On step 10010 episode reward 11.0
On step 10045 episode reward 35.0
On step 10055 episode reward 10.0
On step 10105 episode reward 50.0
On step 10116 episode reward 11.0
On step 10135 episode reward 19.0
On step 10146 episode reward 11.0
On step 10176 episode reward 30.0
On step 10185 episode reward 9.0
On step 10221 episode reward 36.0
On step 10232 episode reward 11.0
On step 10252 episode reward 20.0
On step 10265 episode reward 13.0
On step 10281 episode reward 16.0
On step 10290 episode reward 9.0
On step 10314 episode reward 24.0
On step 10323 episode reward 9.0
On step 10356 episode reward 33.0
On step 10379 episode reward 23.0
On step 10392 episode reward 13.0
On step 10407 episode reward 15.0
On step 10430 episode reward 23.0
On step 10443 episode reward 13.0
On step 10459 episode reward 16.0
On step 10473 episode reward 14.0
On step 10488 episode reward 15.0
On step 10504 episode reward 16.0
On step 10521 episode reward 17.0
On step 10595 episode reward 74.0
On step 10605 epi

On step 15049 episode reward 57.0
On step 15090 episode reward 41.0
On step 15112 episode reward 22.0
On step 15122 episode reward 10.0
On step 15135 episode reward 13.0
On step 15152 episode reward 17.0
On step 15164 episode reward 12.0
On step 15178 episode reward 14.0
On step 15189 episode reward 11.0
On step 15204 episode reward 15.0
On step 15233 episode reward 29.0
On step 15246 episode reward 13.0
On step 15265 episode reward 19.0
On step 15277 episode reward 12.0
On step 15318 episode reward 41.0
On step 15330 episode reward 12.0
On step 15347 episode reward 17.0
On step 15362 episode reward 15.0
On step 15384 episode reward 22.0
On step 15414 episode reward 30.0
On step 15425 episode reward 11.0
On step 15439 episode reward 14.0
On step 15451 episode reward 12.0
On step 15468 episode reward 17.0
On step 15481 episode reward 13.0
On step 15496 episode reward 15.0
On step 15514 episode reward 18.0
On step 15527 episode reward 13.0
On step 15540 episode reward 13.0
On step 15559 

On step 19493 episode reward 16.0
On step 19507 episode reward 14.0
On step 19536 episode reward 29.0
On step 19552 episode reward 16.0
On step 19573 episode reward 21.0
On step 19587 episode reward 14.0
On step 19602 episode reward 15.0
On step 19615 episode reward 13.0
On step 19628 episode reward 13.0
On step 19641 episode reward 13.0
On step 19656 episode reward 15.0
On step 19670 episode reward 14.0
On step 19687 episode reward 17.0
On step 19703 episode reward 16.0
On step 19722 episode reward 19.0
On step 19741 episode reward 19.0
On step 19760 episode reward 19.0
On step 19772 episode reward 12.0
On step 19782 episode reward 10.0
On step 19799 episode reward 17.0
On step 19814 episode reward 15.0
On step 19833 episode reward 19.0
On step 19848 episode reward 15.0
On step 19857 episode reward 9.0
On step 19868 episode reward 11.0
On step 19878 episode reward 10.0
On step 19889 episode reward 11.0
On step 19919 episode reward 30.0
On step 19927 episode reward 8.0
On step 19944 ep

On step 24338 episode reward 12.0
On step 24358 episode reward 20.0
On step 24371 episode reward 13.0
On step 24390 episode reward 19.0
On step 24403 episode reward 13.0
On step 24425 episode reward 22.0
On step 24454 episode reward 29.0
On step 24474 episode reward 20.0
On step 24500 episode reward 26.0
On step 24513 episode reward 13.0
On step 24531 episode reward 18.0
On step 24542 episode reward 11.0
On step 24552 episode reward 10.0
On step 24566 episode reward 14.0
On step 24580 episode reward 14.0
On step 24590 episode reward 10.0
On step 24612 episode reward 22.0
On step 24624 episode reward 12.0
On step 24649 episode reward 25.0
On step 24666 episode reward 17.0
On step 24680 episode reward 14.0
On step 24693 episode reward 13.0
On step 24711 episode reward 18.0
On step 24762 episode reward 51.0
On step 24778 episode reward 16.0
On step 24793 episode reward 15.0
On step 24802 episode reward 9.0
On step 24834 episode reward 32.0
On step 24845 episode reward 11.0
On step 24862 e

On step 29298 episode reward 17.0
On step 29309 episode reward 11.0
On step 29326 episode reward 17.0
On step 29349 episode reward 23.0
On step 29373 episode reward 24.0
On step 29391 episode reward 18.0
On step 29405 episode reward 14.0
On step 29420 episode reward 15.0
On step 29431 episode reward 11.0
On step 29469 episode reward 38.0
On step 29491 episode reward 22.0
On step 29510 episode reward 19.0
On step 29524 episode reward 14.0
On step 29536 episode reward 12.0
On step 29560 episode reward 24.0
On step 29575 episode reward 15.0
On step 29599 episode reward 24.0
On step 29611 episode reward 12.0
On step 29642 episode reward 31.0
On step 29658 episode reward 16.0
On step 29671 episode reward 13.0
On step 29683 episode reward 12.0
On step 29693 episode reward 10.0
On step 29705 episode reward 12.0
On step 29734 episode reward 29.0
On step 29744 episode reward 10.0
On step 29758 episode reward 14.0
On step 29772 episode reward 14.0
On step 29785 episode reward 13.0
On step 29800 

On step 36850 episode reward 103.0
On step 36933 episode reward 83.0
On step 37020 episode reward 87.0
On step 37038 episode reward 18.0
On step 37150 episode reward 112.0
On step 37171 episode reward 21.0
On step 37226 episode reward 55.0
On step 37243 episode reward 17.0
On step 37272 episode reward 29.0
On step 37301 episode reward 29.0
On step 37319 episode reward 18.0
On step 37337 episode reward 18.0
On step 37353 episode reward 16.0
On step 37381 episode reward 28.0
On step 37400 episode reward 19.0
On step 37522 episode reward 122.0
On step 37555 episode reward 33.0
On step 37579 episode reward 24.0
On step 37609 episode reward 30.0
On step 37628 episode reward 19.0
On step 37645 episode reward 17.0
On step 37716 episode reward 71.0
On step 37776 episode reward 60.0
On step 37809 episode reward 33.0
On step 37861 episode reward 52.0
On step 37893 episode reward 32.0
On step 37901 episode reward 8.0
On step 37913 episode reward 12.0
On step 37924 episode reward 11.0
On step 3796

KeyboardInterrupt: 

In [13]:
for episode in range(10):
    episode_reward=0
    curr_state=env.reset()
    done=False
    while not done:
        env.render()
        act=getAction(curr_state,0.01)
        next_state,reward,done,info=env.step(act)
        episode_reward+=reward
        curr_state=next_state
    print("On episode {} reward {}".format(episode,episode_reward))
env.close()

On episode 0 reward 200.0
On episode 1 reward 200.0
On episode 2 reward 197.0
On episode 3 reward 200.0
On episode 4 reward 200.0
On episode 5 reward 200.0
On episode 6 reward 200.0
On episode 7 reward 200.0
On episode 8 reward 200.0
On episode 9 reward 200.0
